In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os

base_project_dir = os.getcwd()
dataset_root_folder = os.path.join(base_project_dir, 'dataset')

# Define the direct paths to the training and test image folders
train_dir = os.path.join(dataset_root_folder, 'training_set', 'training_set')
test_dir = os.path.join(dataset_root_folder, 'test_set', 'test_set')

print(f"Expecting Training data in: {train_dir}")
print(f"Expecting Test data in: {test_dir}")

# Basic check if the primary dataset folder exists
if not os.path.exists(dataset_root_folder):
    print(f"Error: The 'dataset' folder was not found at {dataset_root_folder}")
    print("Please create it and ensure your data is extracted inside it according to the specified structure.")
    exit()

# Model and Training Parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 17

# --- Create Datasets ---
print("\nCreating training dataset...")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)

print("Creating validation dataset...")
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42,
    validation_split=0.2,
    subset='validation'
)

class_names = train_dataset.class_names
print(f"Class names found: {class_names}")
if len(class_names) == 2:
    print(f"Labels will be 0 for '{class_names[0]}' and 1 for '{class_names[1]}'")
else:
    print(f"Warning: Expected 2 classes for binary classification, but found {len(class_names)}: {class_names}")
    print("The model is configured for binary classification. Please check your dataset structure in:")
    print(f"- Training: {train_dir}")
    print(f"- Test: {test_dir}")


AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Define the Model
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# --- Compile the Model ---
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# Train the Model
print("\nStarting model training...")
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)
print("Training finished.")

# Evaluate on the Test Set
print("\nCreating test dataset ...")
test_images_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False
)
test_images_dataset = test_images_dataset.prefetch(buffer_size=AUTOTUNE)

print("\nEvaluating on the dedicated test set:")
loss, accuracy = model.evaluate(test_images_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


# Save the Model
model_save_path = 'cat_dog_classifier.keras'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# Show Training History
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss_hist = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss_hist, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

print("\nFinished.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os
import glob # To find image files

# --- 1. Load the Trained Model ---
MODEL_NAME = 'cat_dog_classifier.keras'
try:
    model = tf.keras.models.load_model(MODEL_NAME)
    print(f"Model '{MODEL_NAME}' loaded successfully.")
except Exception as e:
    print(f"Error loading model '{MODEL_NAME}': {e}")
    exit()

# Image Parameters and Class Names
IMG_HEIGHT = 128
IMG_WIDTH = 128

class_names_pred = ['cats', 'dogs']
print(f"Using class names for prediction: {class_names_pred}")
print(f"This means: Class 0 is '{class_names_pred[0]}', Class 1 is '{class_names_pred[1]}'")

# Prediction Function
# score < 0.5 means the model predicts class 0
# score >= 0.5 means the model predicts class 1
def predict_image(image_path, model_to_use, class_labels):
    try:
        img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return

    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model_to_use.predict(img_array)
    score = predictions[0][0]

    plt.imshow(img)
    plt.axis('off')

    if score < 0.5: # More likely to be class 0
        predicted_class_name = class_labels[0]
        confidence = 1 - score
    else: # More likely to be class 1
        predicted_class_name = class_labels[1]
        confidence = score

    plt.title(f"Prediction: {predicted_class_name} ({confidence*100:.2f}%)")
    plt.show()

    print(f"Image: {os.path.basename(image_path)}")
    print(f"Raw prediction score (prob of being '{class_labels[1]}'): {score:.4f}")
    print(f"This image is predicted as: {predicted_class_name} with {confidence*100:.2f}% confidence.")
    print("-" * 30)


prediction_folder_path = os.path.join(os.getcwd(), 'predict-images')

if not os.path.exists(prediction_folder_path):
    print(f"\nFolder '{prediction_folder_path}' not found.")
    print("Please create it and add images (.jpg, .jpeg, .png) you want to predict.")
else:
    print(f"\n--- Looking for images in: {prediction_folder_path} ---")
    # Supported image extensions
    image_extensions = ["*.jpg", "*.jpeg", "*.png"]
    image_files_to_predict = []
    for ext in image_extensions:
        image_files_to_predict.extend(glob.glob(os.path.join(prediction_folder_path, ext)))

    if not image_files_to_predict:
        print(f"No images found in '{prediction_folder_path}'.")
        print("Please add images (.jpg, .jpeg, .png) to this folder.")
    else:
        print(f"Found {len(image_files_to_predict)} image(s) to predict.")
        for img_file_path in image_files_to_predict:
            predict_image(img_file_path, model, class_names_pred)

print("\nPrediction script finished.")